In [1]:
import os
import mysql.connector as connector
import logging

In [2]:
logger = logging.getLogger("[CTE MySQL]")
if os.path.exists("../log/cte-mysql.log"):
  os.remove("../log/cte-mysql.log")
logging.basicConfig(filename='../log/cte-mysql.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s ==> %(message)s', datefmt='%m/%d/%Y %I:%M:%S')

In [3]:
logger.info("Creating a connection between MySQL and Python")
dbconfig={"user":"root", "password":os.environ["MYSQL_PASSWORD"], "port":33061, "host":"localhost", "database": "db_subqueries"}
connection=connector.connect(**dbconfig)
print("Connection established between MySQL and Python")
logger.info("Connection established between MySQL and Python")

Connection established between MySQL and Python


In [4]:
print("Creating cursor object from connection")
logger.info("Creating first cursor object from connection")
cursor = connection.cursor()
print("Cursor object created to communicate with MySQL using Python.")
logger.info("Cursor object created to communicate with MySQL using Python.")

Creating cursor object from connection
Cursor object created to communicate with MySQL using Python.


In [5]:
def select_all_query(table_name: str):
    query = f"""SELECT * FROM {table_name};"""
    return query


def display_results(table_column_names: list, results: list):
    table_columns_length = [len(x) for x in table_column_names]
    for result in results:
        for value in range(len(result)):
            row_data = result[value]
            if row_data:
                row_data = str(row_data)
                if len(row_data) > table_columns_length[value]:
                    table_columns_length[value] = len(row_data)
    dashes_plus = ""
    for num in range(len(table_columns_length)):
        dashes_plus = dashes_plus + "+" + '-'*(table_columns_length[num]+2)
    dashes_plus = dashes_plus + "+"
    
    print(dashes_plus)
    
    table_headers = ""
    for num in range(len(table_column_names)):
        table_headers = table_headers + f"| {table_column_names[num]:^{table_columns_length[num]}} "
    table_headers = table_headers + "|"
    print(table_headers)
    
    print(dashes_plus)
    
    for result in results:
        table_row = ""
        for value in range(len(result)):
            row_data = result[value]
            if row_data is None:
                row_data = "NULL"            
                #if "Field" in table_column_names or "select_type" in table_column_names:
                    #row_data = None
                #else:
                    #row_data = None
            table_row = table_row + "|" + f"{str(row_data):^{table_columns_length[value]+2}}"
        print(table_row + "|")
    print(dashes_plus)
    print(f"{len(results)} rows returned")

def execute_display_query_results(query: str = "", table_column_names: list = [], results: list = []): 
    if query:
        logger.info(f"Executing the query: {query}")
    if len(query) > 2 and (table_column_names or results):
        print("You can only pass in the query alone or the table_column_names and results list")
        assert False
    if query and not table_column_names and not results:
        cursor.execute(query)
        results = cursor.fetchall()    
        table_column_names = cursor.column_names
    
    display_results(table_column_names, results)

In [6]:
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
for table in tables:
    print(table[0])
    execute_display_query_results(select_all_query(table[0]))
    print("\n")

tbl_customers
+------------+-----------+----------+---------+-------+
| CustomerID | FirstName | LastName | Country | Score |
+------------+-----------+----------+---------+-------+
|     1      |  Jossef   | Goldberg | Germany |  350  |
|     2      |   Kevin   |  Brown   |   USA   |  900  |
|     3      |   Mary    |   NULL   |   USA   |  750  |
|     4      |   Mark    | Schwarz  | Germany |  500  |
|     5      |   Anna    |  Adams   |   USA   | NULL  |
+------------+-----------+----------+---------+-------+
5 rows returned


tbl_employees
+------------+-----------+----------+------------+------------+--------+--------+-----------+
| EmployeeID | FirstName | LastName | Department | BirthDate  | Gender | Salary | ManagerID |
+------------+-----------+----------+------------+------------+--------+--------+-----------+
|     1      |   Frank   |   Lee    | Marketing  | 1988-12-05 |   M    | 55000  |   NULL    |
|     2      |   Kevin   |  Brown   | Marketing  | 1972-11-25 |   M    | 6

## Common Table Expression
This are temporary, named result set (virtual table) that can be used multiple times within your query to simplify and organize complex query

- Standalone CTE: Defined and used independently. Runs independently as it's self-contained and doesn't rely on other CTEs or queries
- Nested CTE
  
```sql
WITH CTE-NAME AS
(
   SELECT *
   FROM tbl_name
   WHERE 
)
```


```sql
SELECT ... FROM CTE-NAME 
WHERE.....
```

## Task 1 
Find the total sales per customer

In [7]:
select_query = "SELECT CustomerID, SUM(Sales) AS TotalSales FROM tbl_orders GROUP BY CustomerID"
execute_display_query_results(select_query)

+------------+------------+
| CustomerID | TotalSales |
+------------+------------+
|     2      |     55     |
|     3      |    125     |
|     1      |    110     |
|     4      |     90     |
+------------+------------+
4 rows returned


In [8]:
## Order BY is not allowed in the CTE
cte_select_query = """WITH CTE_Total_Sales AS 
(
SELECT CustomerID, SUM(Sales) AS TotalSales 
FROM tbl_orders 
GROUP BY CustomerID
)

SELECT c.CustomerID, c.FirstName, c.LastName, cts.TotalSales FROM tbl_customers AS c LEFT JOIN CTE_Total_Sales cts ON c.CustomerID = cts.CustomerID;
"""
execute_display_query_results(cte_select_query)

+------------+-----------+----------+------------+
| CustomerID | FirstName | LastName | TotalSales |
+------------+-----------+----------+------------+
|     1      |  Jossef   | Goldberg |    110     |
|     2      |   Kevin   |  Brown   |     55     |
|     3      |   Mary    |   NULL   |    125     |
|     4      |   Mark    | Schwarz  |     90     |
|     5      |   Anna    |  Adams   |    NULL    |
+------------+-----------+----------+------------+
5 rows returned


## Task 2
Find the last order date for each customer

In [10]:
cte_select_query = """WITH CTE_Total_Sales AS 
(
SELECT CustomerID, SUM(Sales) AS TotalSales 
FROM tbl_orders 
GROUP BY CustomerID
),

CTE_Last_Order AS (
SELECT CustomerID, MAX(OrderDate) AS Last_Order
FROM tbl_orders
GROUP BY CustomerID
)

SELECT c.CustomerID, c.FirstName, c.LastName, cts.TotalSales, clo.last_Order FROM tbl_customers AS c 
LEFT JOIN CTE_Total_Sales cts ON c.CustomerID = cts.CustomerID
LEFT JOIN CTE_Last_Order AS clo ON c.CustomerID = clo.CustomerID;
"""
execute_display_query_results(cte_select_query)

+------------+-----------+----------+------------+------------+
| CustomerID | FirstName | LastName | TotalSales | last_Order |
+------------+-----------+----------+------------+------------+
|     1      |  Jossef   | Goldberg |    110     | 2025-02-15 |
|     2      |   Kevin   |  Brown   |     55     | 2025-03-10 |
|     3      |   Mary    |   NULL   |    125     | 2025-03-15 |
|     4      |   Mark    | Schwarz  |     90     | 2025-02-18 |
|     5      |   Anna    |  Adams   |    NULL    |    NULL    |
+------------+-----------+----------+------------+------------+
5 rows returned


In [11]:
cte_select_query = """WITH CTE_Total_Sales AS 
(
SELECT CustomerID, SUM(Sales) AS TotalSales 
FROM tbl_orders 
GROUP BY CustomerID
),

CTE_Last_Order AS (
SELECT CustomerID, MAX(OrderDate) AS Last_Order
FROM tbl_orders
GROUP BY CustomerID
)

SELECT tbl_customers.CustomerID, FirstName, LastName, TotalSales, last_Order FROM tbl_customers  
LEFT JOIN CTE_Total_Sales USING(CustomerID)
LEFT JOIN CTE_Last_Order USING(CustomerID);
"""
execute_display_query_results(cte_select_query)

+------------+-----------+----------+------------+------------+
| CustomerID | FirstName | LastName | TotalSales | last_Order |
+------------+-----------+----------+------------+------------+
|     1      |  Jossef   | Goldberg |    110     | 2025-02-15 |
|     2      |   Kevin   |  Brown   |     55     | 2025-03-10 |
|     3      |   Mary    |   NULL   |    125     | 2025-03-15 |
|     4      |   Mark    | Schwarz  |     90     | 2025-02-18 |
|     5      |   Anna    |  Adams   |    NULL    |    NULL    |
+------------+-----------+----------+------------+------------+
5 rows returned
